In [2]:
import os
import pandas as pd
import unicodedata

### TODO: signal data와 meta data (발전용량, 준공년도 등) 이름 통일하기

- list_value_keys는 호기 여러 개 존재하는 발전소여서 호기별로 분리해서 저장하기

- exception
    * 안동소내, 삼척사택 데이터는 일별 발전량만 있고 시간별 데이터 없어서 사용 못할듯

#### 1. 수동 mapping table 
- key: signal data에서의 발전소 이름
- value: meta data에서의 발전소 이름

In [93]:
# Creating a mapping dictionary between file names and actual power plant names
solar_name_mapping = {
    '감우리': '감우리',
    '남제주소내': '남제주소내태양광',
    '무릉리': '무릉리',
    '부산복합자재창고': '부산자재창고태양광',
    '부산본부': ['부산소내태양광#1', '부산소내태양광#2'],
    '부산수처리장': '부산 수처리장 태양광',
    '부산신항': '부산신항태양광',
    '부산역선상주차장': '부산역 선상 주차장',
    '부산운동장': '부산 운동장부지 태양광',
    #'삼척사택': '삼척사택 태양광',
    '삼척소내': ['삼척 소내태양광 #1', '삼척 소내태양광 #2', '삼척 소내태양광 #3', '삼척소내#4'],
    '세화리': '세화리',
    '송당리': '송당리 태양광',
    '신인천 1_2단계 주차장': '신인천 1,2단계 주차장 태양광',
    '신인천 북측부지': '신인천 북측부지 태양광',
    '신인천 주차장': '신인천 주차장 태양광',
    '신인천본관주차장': '신인천 소내주차장',
    '신인천소내': '신인천소내태양광',
    '신인천전망대': '신인천 전망대 태양광',
    '신인천해수구취수구': '신인천해수취수로태양광',
    '신풍리': '신풍리',
    #'안동소내': ['안동 소내태양광 #1', '안동 소내태양광#2'],
    '영월본부': '영월복합태양광',
    '영월철도부지': '영월 철도부지 태양광',
    '와산리': '와산리',
    '용수리': '제주 용수리 태양광',
    '위미2리': '위미2리태양광',
    '이천D(백사면B)': '백사면 B',
    '이천시 백사면A': '백사면 A',
    '익산 다송리': '익산다송리',
    '인천수산정수장': '인천수산정수장태양광',
    '하동공설운동장': '하동공설운동장태양광',
    '하동변전소': '하동변전소태양광',
    '하동보건소': '하동군보건소태양광',
    '하동본부': ['하동화력태양광 #1', '하동화력태양광 #2 (3개소)', '하동화력태양광 #3', '하동화력태양광 #4', '하동화력 태양광 #5', '하동소내#6'],
    '하동정수장': '하동정수장태양광',
    '하동하수처리장': '하동하수처리장태양광',
    '행원소수력': '행원소수력태양광',
    '화촌주민참여형': '화촌 솔라팜'
}
solar_name_mapping = {
    unicodedata.normalize('NFC', str(k).strip()): v 
    for k, v in solar_name_mapping.items()
}
len(solar_name_mapping)

37

In [94]:
# 데이터 저장할 경로 정의
save_path = '../data/processed_data'
os.makedirs(save_path,exist_ok=True)

#### 2. 필요한 Meta data만 solar_name_mapping의 key 활용하여 table로 저장하기

In [95]:
# Signal files
path = '../data/raw_data'
files = [i for i in os.listdir(path) if '태양광' in i]
files = [
    unicodedata.normalize('NFC', name.strip())
    for name in files
]
files = [f for f in files if '중부발전' not in f]
file_names = [name.split('(주)_')[1].split('태양광')[0].strip() for name in files]
file_names = [
    unicodedata.normalize('NFC', name.strip())
    for name in file_names
]
file_mapping_dict = {}
for i in range(len(files)):
    file_mapping_dict[file_names[i]] = files[i]

# Meta data
meta_data = pd.read_csv(os.path.join(path, '한국남부발전(주)_에너지원별 신재생에너지 발전설비 현황_20250312.csv'),encoding='cp949')
meta_data['발전소명'] = meta_data['발전소명'].astype(str).str.strip()
meta_data['발전소명'] = meta_data['발전소명'].apply(lambda x: unicodedata.normalize('NFC', x))


file_names.sort()
print("file_names:",len(file_names))
print(file_names[:4],'..')

file_names: 39
['감우리', '남제주소내', '무릉리', '부산복합자재창고'] ..


In [96]:
print('하나의 signal file에 meta data는 여러 개 존재하는 경우 (발전호기가 2개 이상인 경우)')
list_value_keys = [k for k, v in solar_name_mapping.items() if isinstance(v, list)]
list_value_keys

하나의 signal file에 meta data는 여러 개 존재하는 경우 (발전호기가 2개 이상인 경우)


['부산본부', '삼척소내', '하동본부']

In [97]:
all_meta_data = []

# 1. 1:1 매핑 가능한 데이터들에 대해서 우선 저장
keys = [i for i in solar_name_mapping.keys() if i not in list_value_keys]
print('1. 1:1 매핑 가능한 데이터들:',len(keys),'\n')

for key in keys:
    meta = meta_data[meta_data['발전소명']==solar_name_mapping[key]].copy()
    meta['name'] = key
    all_meta_data.append(meta[['name','발전소명','구분','용량(MW)','준공년도','비 고']] )

import re
# 2. 발전호기 2개 이상인 경우를 처리
print('2. 발전호기 2개 이상인 경우:',len(list_value_keys))
print('-> name_number 형식으로 name 저장함')
for key in list_value_keys:
    meta = meta_data[meta_data['발전소명'].isin(solar_name_mapping[key])].copy()
    # 발전소명 안의 숫자 추출 & name 생성
    names = []
    for plant_name in meta['발전소명']:
        match = re.search(r'#\s*(\d+)', plant_name)  # '# 숫자' 패턴 찾기
        if match:
            num = match.group(1)
            new_name = f"{key}_{num}"
        else:
            new_name = key  # 숫자가 없으면 그냥 key 사용
        names.append(new_name)

    meta['name'] = names

    all_meta_data.append(meta[['name', '발전소명', '구분', '용량(MW)', '준공년도', '비 고']])
all_meta_data = pd.concat(all_meta_data,ignore_index=True)
all_meta_data.to_csv(os.path.join(save_path,'meta_data.csv'),index=False)
all_meta_data


1. 1:1 매핑 가능한 데이터들: 34 

2. 발전호기 2개 이상인 경우: 3
-> name_number 형식으로 name 저장함


,name,발전소명,구분,용량(MW),준공년도,비 고
0,감우리,감우리,자체,0.5540,2021,RPS
1,남제주소내,남제주소내태양광,자체,0.1960,2012,RPS
2,무릉리,무릉리,자체,0.8090,2021,RPS
3,부산복합자재창고,부산자재창고태양광,자체,0.1870,2012,RPS
4,부산수처리장,부산 수처리장 태양광,자체,0.1110,2017,RPS
5,부산신항,부산신항태양광,자체,0.1150,2011,RPA
6,부산역선상주차장,부산역 선상 주차장,자체,0.8344,2020,RPS
7,부산운동장,부산 운동장부지 태양광,자체,1.4000,2017,RPS
8,세화리,세화리,자체,0.9990,2021,RPS
9,송당리,송당리 태양광,자체,0.9970,2019,RPS


#### 3. signal data도 solar_name_mapping key 이름으로 저장해두기

- col name '14-15시 발전량(kWh)': 세화리, 신인천본관주차장,와산리,익산 다송리
    * 이 case들에 대해서는 추가로 col name 수정해서 저장함

In [101]:
# 1. 1:1 매핑 가능한 데이터들
for key in keys:
    choose = file_mapping_dict[key]
    file = pd.read_csv(os.path.join(path,choose),encoding='cp949')
    try:
        # 보통의 데이터 시간대별 column name: 1,2,3,4..
        _=  file['15']
    except:
        # col name '14-15시 발전량(kWh)': 세화리, 신인천본관주차장,와산리,익산 다송리
        # 시간대 컬럼만 찾아서 숫자형으로 바꾸기
        rename_dict = {}
        for col in file.columns:
            if isinstance(col, str) and '시 발전량' in col:
                # ex) '14-15시 발전량 (kWh)' → 15
                hour = col.split('-')[-1].split('시')[0].strip()
                rename_dict[col] = hour
        file.rename(columns=rename_dict, inplace=True)
    
    file.to_csv(os.path.join(save_path,f'{key}.csv'),index=False)

# 2. 발전호기 2개 이상인 경우
def save_signal_per_unit(file, key, save_path):
    # unique 호기 번호 찾기
    unique_units = file['호기'].unique()

    for unit in unique_units:
        # 해당 호기의 데이터만 추출
        unit_df = file[file['호기'] == unit].copy()

        # 년월일로 정렬
        unit_df = unit_df.sort_values(by='년월일').reset_index(drop=True)

        # 파일 이름: key_호기.csv
        file_name = f"{key}_{unit}.csv"
        unit_df.to_csv(os.path.join(save_path, file_name), index=False, encoding='cp949')  # 한글 호환용 cp949
        #print(f"Saved: {file_name}")

for key in list_value_keys:
    choose = file_mapping_dict[key]
    file = pd.read_csv(os.path.join(path,choose),encoding='cp949')
    try:
        # 보통의 데이터 시간대별 column name: 1,2,3,4..
        _=  file['15']
    except:
        # col name '14-15시 발전량(kWh)': 세화리, 신인천본관주차장,와산리,익산 다송리
        # 시간대 컬럼만 찾아서 숫자형으로 바꾸기
        rename_dict = {}
        for col in file.columns:
            if isinstance(col, str) and '시 발전량' in col:
                # ex) '14-15시 발전량 (kWh)' → 15
                hour = col.split('-')[-1].split('시')[0].strip()
                rename_dict[col] = hour
        file.rename(columns=rename_dict, inplace=True)
    save_signal_per_unit(file, key, save_path)

In [104]:
# 확인
file = pd.read_csv(os.path.join(save_path,'세화리.csv'),encoding='utf8')
file

,년월일,호기,설비용량(kW),총량(kWh),평균(kWh),최대(kWh),최소(kWh),최대(시간별_kWh),최소(시간별_kWh),1,...,15,16,17,18,19,20,21,22,23,24
0,2021-12-02,1,999.6,2247.574,93.649,397.014,0,397.014,1.446,0,...,200.315,151.863,37.604,4.339,0.000,0.000,0,0,0,0
1,2021-12-03,1,999.6,3668.580,152.858,734.005,0,734.005,0.723,0,...,519.951,225.625,26.034,0.723,0.000,0.000,0,0,0,0
2,2021-12-04,1,999.6,4014.249,167.260,707.248,0,707.248,5.062,0,...,554.662,342.777,103.412,5.062,0.000,0.000,0,0,0,0
3,2021-12-05,1,999.6,1658.922,69.122,344.946,0,344.946,1.446,0,...,174.281,216.947,65.084,1.446,0.000,0.000,0,0,0,0
4,2021-12-06,1,999.6,4011.355,167.140,734.728,0,734.728,1.446,0,...,583.588,57.853,31.819,1.446,0.000,0.000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,2024-06-26,1,999.6,2768.249,115.344,405.692,0,405.692,0.723,0,...,373.149,360.856,366.641,155.479,33.988,3.616,0,0,0,0
938,2024-06-27,1,999.6,439.681,18.320,83.164,0,83.164,0.723,0,...,44.113,83.164,31.819,27.480,4.339,0.000,0,0,0,0
939,2024-06-28,1,999.6,1601.071,66.711,214.055,0,214.055,2.169,0,...,125.830,214.055,151.140,68.700,22.418,2.169,0,0,0,0
940,2024-06-29,1,999.6,118.596,4.942,23.141,0,23.141,0.723,0,...,23.141,23.141,2.892,1.446,0.000,0.000,0,0,0,0


#### 안씀

In [28]:

# def create_solar_mapping(file_names, meta_data):
#     """
#     파일 이름과 메타데이터의 발전소명을 매핑하는 함수

#     Parameters:
#     file_names (list): 파일 이름 리스트
#     meta_data (DataFrame): 발전소명이 포함된 메타데이터 DataFrame

#     Returns:
#     dict: 파일 이름을 키로, 매칭되는 발전소명(들)을 값으로 하는 사전
#     """
#     # 태양광 발전소 이름만 추출
#     solar_plants = meta_data[meta_data['에너지원'] == '태양광']['발전소명'].unique()

#     # 띄어쓰기 제거 및 소문자 변환을 통한 정규화 함수
#     def normalize_string(s):
#         return s.replace(' ', '').lower()

#     # 결과 사전 초기화
#     solar_mapping = {}

#     # 각 파일 이름에 대해 매칭되는 발전소명 찾기
#     for file_name in file_names:
#         normalized_file_name = normalize_string(file_name)
#         matches = []

#         for plant in solar_plants:
#             normalized_plant = normalize_string(plant)

#             # 파일명이 발전소명에 포함되어 있거나, 발전소명이 파일명에 포함되어 있는 경우
#             if normalized_file_name in normalized_plant or normalized_plant in normalized_file_name:
#                 matches.append(plant)

#         # 매칭 결과 저장
#         if len(matches) == 1:
#             solar_mapping[file_name] = matches[0]  # 단일 매칭
#         elif len(matches) > 1:
#             solar_mapping[file_name] = matches  # 다중 매칭은 리스트로 저장
#         else:
#             solar_mapping[file_name] = None  # 매칭 실패

#     return solar_mapping

# # 코드 사용 예시
# solar_mapping = create_solar_mapping(file_names, meta_data)
# print(solar_mapping)

# # 매핑 결과 확인 및 수정이 필요한 항목 확인
# def check_mapping_results(solar_mapping):
#     """
#     매핑 결과를 확인하고 문제점을 보고하는 함수

#     Parameters:
#     solar_mapping (dict): 매핑 결과 사전

#     Returns:
#     tuple: (매핑 성공 항목 수, 매핑 실패 항목 수, 다중 매핑 항목 수)
#     """
#     success_count = 0
#     failed_count = 0
#     multiple_count = 0

#     print("\n===== 매핑 결과 요약 =====")

#     print("\n[매핑 실패 항목]")
#     for file_name, plant in solar_mapping.items():
#         if plant is None:
#             failed_count += 1
#             print(f"- {file_name}: 매칭 실패")

#     print("\n[다중 매핑 항목]")
#     for file_name, plant in solar_mapping.items():
#         if isinstance(plant, list):
#             multiple_count += 1
#             print(f"- {file_name}: {len(plant)}개 매칭 {plant}")

#     print("\n[단일 매핑 항목]")
#     for file_name, plant in solar_mapping.items():
#         if plant is not None and not isinstance(plant, list):
#             success_count += 1
#             print(f"- {file_name} -> {plant}")

#     print(f"\n총 {len(solar_mapping)}개 항목 중:")
#     print(f"- 성공: {success_count}개")
#     print(f"- 실패: {failed_count}개")
#     print(f"- 다중 매칭: {multiple_count}개")

#     return success_count, failed_count, multiple_count

# # 매핑 결과 확인 사용 예시
# check_mapping_results(solar_mapping)

# # 다중 매핑 결과 수동 수정 함수
# def fix_multiple_mappings(solar_mapping, manual_fixes):
#     """
#     다중 매핑 결과를 수동으로 수정하는 함수

#     Parameters:
#     solar_mapping (dict): 원본 매핑 결과 사전
#     manual_fixes (dict): 수정할 항목 사전 {파일명: 선택할_발전소명}

#     Returns:
#     dict: 수정된 매핑 결과 사전
#     """
#     fixed_mapping = solar_mapping.copy()

#     for file_name, correct_plant in manual_fixes.items():
#         if file_name in fixed_mapping:
#             fixed_mapping[file_name] = correct_plant

#     return fixed_mapping

# # 수동 수정 예시
# # manual_fixes = {
# #     '신인천': '신인천소내태양광',  # 여러 '신인천' 중 하나를 선택
# #     '삼척소내': '삼척 소내태양광 #1'  # 여러 버전 중 하나를 선택
# # }
# #fixed_solar_mapping = fix_multiple_mappings(solar_mapping, manual_fixes)

{'부산수처리장': '부산 수처리장 태양광', '익산 다송리': '익산다송리', '부산복합자재창고': None, '삼척사택': '삼척사택 태양광', '와산리': '와산리', '하동보건소': None, '하동정수장': '하동정수장태양광', '하동하수처리장': '하동하수처리장태양광', '송당리': '송당리 태양광', '화촌주민참여형': None, '신인천해수구취수구': None, '남제주소내': '남제주소내태양광', '영월철도부지': '영월 철도부지 태양광', '하동본부': None, '신인천본관주차장': None, '인천수산정수장': '인천수산정수장태양광', '신인천 1_2단계 주차장': None, '안동소내': ['안동 소내태양광 #1', '안동 소내태양광#2'], '하동변전소': '하동변전소태양광', '신인천전망대': '신인천 전망대 태양광', '부산신항': '부산신항태양광', '신인천소내': ['신인천소내태양광', '신인천 소내주차장'], '부산운동장': '부산 운동장부지 태양광', '신인천 북측부지': '신인천 북측부지 태양광', '위미2리': '위미2리태양광', '하동공설운동장': '하동공설운동장태양광', '용수리': '제주 용수리 태양광', '신인천 주차장': '신인천 주차장 태양광', '부산본부': None, '영월본부': None, '세화리': '세화리', '무릉리': '무릉리', '삼척소내': ['삼척 소내태양광 #1', '삼척 소내태양광 #2', '삼척 소내태양광 #3', '삼척소내#4'], '부산역선상주차장': '부산역 선상 주차장', '행원소수력': '행원소수력태양광', '이천D(백사면B)': '백사면 B'}

===== 매핑 결과 요약 =====

[매핑 실패 항목]
- 부산복합자재창고: 매칭 실패
- 하동보건소: 매칭 실패
- 화촌주민참여형: 매칭 실패
- 신인천해수구취수구: 매칭 실패
- 하동본부: 매칭 실패
- 신인천본관주차장: 매칭 실패
- 신인천 1_2단계 주차장: 매칭 실패
- 부산본부: 매칭 실패
- 영월본부: 매칭 실패

[